In [1]:
import sys
import os
if 'google.colab' in sys.modules:
    # Google Drive
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
    
    
    # Kaggle API
    !pip install --upgrade --force-reinstall --no-deps kaggle
    import json
    f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
    json_data = json.load(f)
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]
    

    # Output File Location
    dname = "localnb003-spacy-dataset"
    os.makedirs(dname, exist_ok=True)
    
    # GPU
    !nvidia-smi

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
Processing /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303/kaggle-1.5.12-cp37-none-any.whl
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Tue Jun 22 10:35:21 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.27       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|=========================

In [ ]:
# %%time
# !pip uninstall fastai en-core-web-sm en-core-web-lg spacy -y -q
# !pip install ../input/spacy3/catalogue-2.0.3-py3-none-any.whl ../input/spacy3/typer-0.3.2-py3-none-any.whl ../input/spacy3/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/pathy-0.5.2-py3-none-any.whl ../input/spacy3/smart_open-3.0.0-py3-none-any.whl ../input/spacy3/pydantic-1.7.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/thinc-8.0.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy_legacy-3.0.5-py2.py3-none-any.whl -q
# !pip install ../input/spacy3/en_core_web_lg-3.0.0-py3-none-any.whl ../input/spacy3/en_core_web_md-3.0.0-py3-none-any.whl ../input/spacy3/en_core_web_sm-3.0.0-py3-none-any.whl -q
# !pip install ../input/spacy3/spacy_alignments-0.8.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy_transformers-1.0.2-py2.py3-none-any.whl ../input/spacy3/en_core_web_trf-3.0.0-py3-none-any.whl -q

In [3]:
!pip install spacy==3.0.6

     |████████████████████████████████| 12.8MB 8.1MB/s 
     |████████████████████████████████| 624kB 66.5MB/s 
     |████████████████████████████████| 460kB 54.7MB/s 
     |████████████████████████████████| 51kB 8.2MB/s 
     |████████████████████████████████| 9.1MB 38.3MB/s 
     |████████████████████████████████| 122kB 66.9MB/s 
  Created wheel for smart-open: filename=smart_open-3.0.0-cp37-none-any.whl size=107107 sha256=17ebd703dbe9bc258f3b0220b32cd719f29dd619ebae2a0c1aede1f1ab321ffb
  Stored in directory: /root/.cache/pip/wheels/18/88/7c/f06dabd5e9cabe02d2269167bcacbbf9b47d0c0ff7d6ebcb78
Successfully built smart-open
  Found existing installation: catalogue 1.0.0
    Uninstalling catalogue-1.0.0:
      Successfully uninstalled catalogue-1.0.0
  Found existing installation: srsly 1.0.5
    Uninstalling srsly-1.0.5:
      Successfully uninstalled srsly-1.0.5
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found e

In [2]:
import spacy
assert spacy.__version__ == '3.0.6'
from spacy.tokens import DocBin

In [3]:
!python -m spacy download en_core_web_trf

2021-06-22 10:35:44.174216: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 459.7MB 34kB/s 
     |████████████████████████████████| 2.3MB 11.5MB/s 
     |████████████████████████████████| 1.0MB 56.4MB/s 
     |████████████████████████████████| 901kB 54.2MB/s 
     |████████████████████████████████| 3.3MB 53.9MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')


In [4]:
import en_core_web_trf

In [5]:
import re
import json
import time
import datetime
import random
import glob
import importlib
import pickle
import numpy as np
import pandas as pd
import joblib
from joblib import Parallel, delayed

from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import gc 

from sklearn.model_selection import GroupKFold

import torch 
if torch.cuda.is_available():
    import cupy
    spacy.prefer_gpu()
    

In [6]:
def read_json_pub(filename, train_data_path, output='text'):
    json_path = os.path.join(train_data_path, (filename+'.json'))
    headings = []
    contents = []
    combined = []
    with open(json_path, 'r') as f:
        json_decode = json.load(f)
        for data in json_decode:
            headings.append(data.get('section_title'))
            contents.append(data.get('text'))
            combined.append(data.get('section_title'))
            combined.append(data.get('text'))
    
    all_headings = ' '.join(headings)
    all_contents = ' '.join(contents)
    all_data = '. '.join(combined)
    
    if output == 'text':
        return all_contents
    elif output == 'head':
        return all_headings
    else:
        return all_data

In [7]:
sample_sub = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/sample_submission.csv')
train_data_path = '../input/coleridgeinitiative-show-us-the-data/train'
test_data_path = '../input/coleridgeinitiative-show-us-the-data/test'
train_df = pd.read_csv('../input/coleridgeinitiative-show-us-the-data/train.csv')

In [8]:
def text_cleaning(text):
    '''
    Converts all text to lower case, Removes special charecters, emojis and multiple spaces
    text - Sentence that needs to be cleaned
    '''
    text = re.sub('[^A-Za-z0-9]+', ' ', str(text).lower()).strip()
    text = re.sub(' +', ' ', text)
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    return text


In [9]:
def clean_text(txt):
    return re.sub('[^A-Za-z0-9]+', ' ', str(txt).lower()).strip()

In [10]:
df2 = pd.read_csv('../input/bigger-govt-dataset-list/data_set_800.csv')

In [ ]:
folds = pd.read_pickle("./nb009-cv/folds_pubcat.pkl")

In [17]:
# gkf= GroupKFold(n_splits=5)
# group_id=[]
# for train_index, test_index in gkf.split(train_df["Id"],train_df["cleaned_label"],train_df["cleaned_label"]): 
#     #train_id=train_df["Id"][train_index].unique()
#     group_id+=[train_df["Id"][test_index].unique()]

# split_id=[]
# for i in range(5):
#     split_id+=[group_id[i]]
#     for j in range(i+1,5):
#         split_id[i]=np.setdiff1d(split_id[i],group_id[j])

#with open("./split_id", "wb") as f:
#    pickle.dump(split_id,f)

In [50]:
split_id = []
for i in range(5):
    ids = folds[folds["fold"] == i+1]["Id"].values
    split_id.append(ids)

In [ ]:
#split_id を　堀内さん作成のｃｖのidに置き換えて使用してください

In [52]:
def make_train_data(split_id,train_data_path,df2,j):
    path=train_data_path
    column_names = ["Id", "PredictionString"]

    submission = pd.DataFrame(columns = column_names)
    fn_list=[]
    fn_text=[]
    to_append=[]
    train_data=[]
    #for index, row in sample_sub.iterrows():
    for id in tqdm(split_id[j]):
        to_append=[id,'']
        large_string = str(read_json_pub(id,path))
        se=large_string.split(".")
        le_se=len(se)
        for i in range(df2.shape[0]):
            mw= str(df2['title'].values[i])
            train_data+=[(i,{"entities":[i.lower().find(mw),i.lower().find(mw)+len(mw),"DB_label"]}) for i in large_string.split(".") if mw in i.lower()]
            #train_data+=[(".".join(se[max([0,i-1]):min([i+2,le_se+1])]),
            #  {"entities":[".".join(se[max([0,i-1]):min([i+2,le_se+1])]).lower().find(mw),
            #               ".".join(se[max([0,i-1]):min([i+2,le_se+1])]).lower().find(mw)+len(mw),
            #               "DB_label"]})
            # for i in range(le_se)
            # if mw in ".".join(se[max([0,i-1]):min([i+2,le_se+1])]).lower()]
            
    return train_data

In [55]:
#cv_numにcv数を入力して使用してください
cv_num=5
train_data_sets = joblib.Parallel(n_jobs=-1, verbose=3)(joblib.delayed(make_train_data)(split_id,train_data_path,df2,j) for j in range(cv_num))

for j in range(cv_num):
    cv_valid=[]
    cv_train=[]
    for i in range(cv_num):
        if i==j :
            cv_valid=train_data_sets[i]
        else:
            cv_train+=train_data_sets[i]
    with open(f"./{dname}/cv_train_"+str(j)+".pkl", "wb") as f:
        pickle.dump(cv_train,f)
    with open(f"./{dname}/cv_valid_"+str(j)+".pkl", "wb") as f:
        pickle.dump(cv_valid,f)


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:  1.4min remaining:  2.1min
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed: 120.6min finished


In [56]:
def has_duplicates(seq1,seq2):
    return len(seq1+seq2) != len(set(seq1+seq2))


def drop_dup_ents(ents):
    e=pd.Series(ents)
    w=[]
    #print(e)
    for i in range(len(e)):
            #s=[str(e[i]) in str(e[j]) for j in range(len(e))] 
            s=[has_duplicates(e[i].text.split(" "),e[j].text.split(" ")) for j in range(len(e))] 
            #s=[has_duplicates(e[i],seq2)
            n0=np.argmax([len(e[s].iloc[k]) for k in range(len(e[s]))])
            w+=[e[s].iloc[n0]]

    w=list(set(w))
    return w



In [57]:
def make_doc(i,train_data):
    db = DocBin()
    tes=[]
    sk=0
    nlp = spacy.blank("en")
    for j in range(1000*i,1000*(i+1)):
      
        text, annot=train_data[j]

        doc = nlp.make_doc(text) # create doc object from text
        ents = []
        start, end, label=annot["entities"]
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span is None:
            #print("Skipping entity")
            sk+=1
        else:
            ents.append(span)
        ents=drop_dup_ents(ents)

        doc.ents = ents # label the text with the ents
        db.add(doc)
    #del doc
    #del ents
    #del nlp
    #gc.collect()
    return db

In [59]:
cv_num

5

In [60]:
for k in range(cv_num):
    with open(f"./{dname}/cv_train_"+str(k)+".pkl", "rb") as f:
        cv_train=pickle.load(f)
    random.shuffle(cv_train)
    l1=int(len(cv_train)/1000)
    result = joblib.Parallel(n_jobs=-1)(joblib.delayed(make_doc)(i,cv_train) for i in tqdm(range(l1)))

    db0=result[0]
    for i in range(1,len(result)):
        db1=result[i]
        db0.merge(db1)
    db0.to_disk(f"./{dname}/cv_train_"+str(k)+".spacy")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


/usr/local/lib/python3.7/dist-packages/joblib/externals/loky/process_executor.py:691: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


In [63]:
for k in range(cv_num):
    with open(f"./{dname}/cv_valid_"+str(k)+".pkl", "rb") as f:
        cv_valid=pickle.load(f)
    random.shuffle(cv_valid)
    l1=int(len(cv_valid)/1000)
    print("l1: ", l1)
    if l1 > 0:
        result = joblib.Parallel(n_jobs=-1)(joblib.delayed(make_doc)(i,cv_valid) for i in tqdm(range(l1)))

        db0=result[0]
        for i in range(1,len(result)):
            db1=result[i]
            db0.merge(db1)
        db0.to_disk(f"./{dname}/cv_valid_"+str(k)+".spacy")
    else:
        print(f"cv_valid_"+str(k)+".spacy could not be made.")
        continue

l1:  36


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  



l1:  40



l1:  3



l1:  0
cv_valid_3.spacy could not be made.
l1:  0
cv_valid_4.spacy could not be made.


In [64]:
dname

'localnb003-spacy-dataset'

In [65]:
!kaggle datasets init -p {dname}
# referene: https://kaeru-nantoka.hatenablog.com/entry/2020/01/17/015551

with open(f"{dname}/dataset-metadata.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["id"] = f"riow1983/{dname}"
data["title"] = dname

with open(f"{dname}/dataset-metadata.json", "w") as jsonFile:
    json.dump(data, jsonFile)

!kaggle datasets create -p {dname}

Data package template written to: localnb003-spacy-dataset/dataset-metadata.json
Starting upload for file cv_train_0.pkl
100% 8.66M/8.66M [00:03<00:00, 2.66MB/s]
Upload successful: cv_train_0.pkl (9MB)
Starting upload for file cv_valid_0.pkl
100% 9.67M/9.67M [00:03<00:00, 3.17MB/s]
Upload successful: cv_valid_0.pkl (10MB)
Starting upload for file cv_train_1.pkl
100% 10.3M/10.3M [00:01<00:00, 5.90MB/s]
Upload successful: cv_train_1.pkl (10MB)
Starting upload for file cv_valid_1.pkl
100% 7.98M/7.98M [00:03<00:00, 2.40MB/s]
Upload successful: cv_valid_1.pkl (8MB)
Starting upload for file cv_train_2.pkl
100% 17.9M/17.9M [00:02<00:00, 7.88MB/s]
Upload successful: cv_train_2.pkl (18MB)
Starting upload for file cv_valid_2.pkl
100% 627k/627k [00:03<00:00, 188kB/s]
Upload successful: cv_valid_2.pkl (627KB)
Starting upload for file cv_train_3.pkl
100% 18.5M/18.5M [00:03<00:00, 6.08MB/s]
Upload successful: cv_train_3.pkl (19MB)
Starting upload for file cv_valid_3.pkl
100% 28.4k/28.4k [00:02<00:00